train, validation은 일단 나눔. 그리고 cbam적용하는 것이 목표임.

In [1]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics
from torch.utils.data import Subset

In [2]:
Catholic_file = 'aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [3]:
df = df.sample(frac=1)

In [4]:
df

,filename,category,class,sex,old
821,minus_0522-1.wav,non-wheezing,0,1,12
975,rs_0023-1.wav,non-wheezing,0,0,4
1095,rs_0550-4.wav,wheezing,1,0,5
196,0597-4.wav,wheezing,1,0,1
659,cs_0600-3.wav,wheezing,1,0,2
...,...,...,...,...,...
164,0526-2.wav,wheezing,1,0,4
412,cp_0581-1.wav,wheezing,1,1,7
113,0041-4.wav,non-wheezing,0,0,2
1008,rs_0034-2.wav,non-wheezing,0,1,2


In [5]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [6]:
df

,filename,category,class,sex,old
821,minus_0522-1.wav,non-wheezing,0,1,2.449728
975,rs_0023-1.wav,non-wheezing,0,0,-0.021489
1095,rs_0550-4.wav,wheezing,1,0,0.287413
196,0597-4.wav,wheezing,1,0,-0.948195
659,cs_0600-3.wav,wheezing,1,0,-0.639293
...,...,...,...,...,...
164,0526-2.wav,wheezing,1,0,-0.021489
412,cp_0581-1.wav,wheezing,1,1,0.905217
113,0041-4.wav,non-wheezing,0,0,-0.639293
1008,rs_0034-2.wav,non-wheezing,0,1,-0.639293


In [7]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
821,/minus_0522-1.wav,0,1,2.449728
975,/rs_0023-1.wav,0,0,-0.021489
1095,/rs_0550-4.wav,1,0,0.287413
196,/0597-4.wav,1,0,-0.948195
659,/cs_0600-3.wav,1,0,-0.639293


In [8]:
data_path = 'aug_train_v2.2'

In [9]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [10]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 22050
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return [aug_sgram, tabular], class_id

In [11]:
from skorch import NeuralNetClassifier

In [12]:
ds = breathDS(df, data_path)

In [13]:
'''num_items = len(brds)
print(len(brds))
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(brds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)'''

'num_items = len(brds)\nprint(len(brds))\nnum_train = round(num_items * 0.8)\nnum_val = num_items - num_train\ntrain_ds, val_ds = random_split(brds, [num_train, num_val])\n\ntrain_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)\nval_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)'

In [14]:
#train_dl = DataLoader(brds, batch_size=16, shuffle=True)

In [15]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [21]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        #self.fc1 = nn.Linear(2, 8)
        #self.fc = nn.Linear((512 * block.expansion)+8, num_classes)
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_tab = nn.Linear(2, 8)
        self.fc_tab2 = nn.Linear(8, 16)
        self.fc = nn.Linear((512 * block.expansion)+16, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

        #self.fc1 = nn.Linear(2, 8)
        #self.fc2 = nn.Linear(8, 16)
        #self.fc = nn.Linear((512 * block.expansion)+16, num_classes)
        #self.fc = nn.Linear((512 * block.expansion), 128)
        #self.fc_ = nn.Linear(160, num_classes)

    def forward(self, x_):
        x = x_[0]
        x2 = x_[1]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #print(x.size())
        #x2 = self.fc1(x2)
        #x2 = self.fc2(x2)
        #print(x2.size())
        #x_ = torch.cat((x1, x2), 1)
        #x = self.fc(x)
        x2 = self.fc_tab(x2)
        x2 = self.fc_tab2(x2)
        #x2 = self.fc2(x2)
        #print(x.size())
        #print(x2.size())
        x = torch.cat((x, x2), 1)
        x = self.fc(x)
        return x

In [22]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from skorch.helper import SliceDataset

Model1 = ResNet(BasicBlock, [2, 2, 2, 2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

params = {
    'lr': [0.001, 0.01],
    'max_epochs': [40, 60, 80],
    'batch_size': [16, 32]
}

net = NeuralNetClassifier(
    Model1,
    optimizer=torch.optim.Adam,
    criterion=nn.CrossEntropyLoss,
    max_epochs=80,
    lr=0.001,
    batch_size=16,
    iterator_train__shuffle=True,
    verbose=1,
    train_split=None
)

In [23]:
x_ = SliceDataset(ds, idx=0)
outputs = SliceDataset(ds, idx=1)

In [24]:
from skorch.helper import SliceDataset

#gs = GridSearchCV(net, params, refit=False, cv=5, scoring='accuracy', error_score="raise", verbose=3, n_jobs=-1)
gg = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
gs = GridSearchCV(net, params, refit=False, cv=gg, scoring='accuracy', error_score="raise", verbose=3)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
gs.fit(x, outputs)

pd.DataFrame(gs.cv_results_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 

torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([2, 512])
torch.Size([2, 2])
[CV 2/5] END batch_size=16, lr=0.001, max_epochs=1;, score=0.860 total time= 1.0min
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16

torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([8, 512])
torch.Size([8, 2])
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4368  33.4471
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch

torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([

torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([16, 2])
torch.Size([16, 512])
torch.Size([